In [1]:
import math
import sys
# 可以从父目录下查找模块
sys.path.append('../')

from model.MDT.MDT6 import mdt20 as train_model
# from model.T2T.T2T import T2t_vit_7 as train_model
# from model.Resnet.resnet import resnet18 as train_model
# from model.Swin.swin_transformer import swin_small_patch4_window7_224 as train_model
# from model.ViT.ViT import vit_base_patch16_224 as train_model
# from model.TNT.tnt import tnt_s_patch16_224 as train_model
# from model.Swin.swin_transformer_v2 import swin_v2 as train_model

import os.path
import time
from datetime import datetime
from openpyxl import Workbook
from tqdm import tqdm
import argparse
import matplotlib.pyplot as plt
import train_utils
from d2l import torch as d2l

import thop
import torch
import torchvision.transforms
import torch.nn as nn

In [2]:
def main(args):
    print(args)
    # 设置设备
    device = torch.device(args.device if torch.cuda.is_available() else 'cpu')
    # 获取设备列表
    devices = d2l.try_all_gpus()
    print('使用设备：{}'.format(devices))
    devices = [d2l.try_gpu(i) for i in range(args.num_gpus)]

    # 数据读取与预处理

    # 训练集 数据归一化
    transform_train = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    # 测试集 数据归一化
    transform_test = torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    train_dataset, test_dataset, train_dataloader, test_dataloader, num_classes = train_utils.data_getter(dataset_root=args.dataset_root,
                                                                                             transform_train=transform_train,
                                                                                             transform_test=transform_test,
                                                                                             batch_size=args.batch_size,
                                                                                             dataset_name=args.dataset_name)


    # 模型加载
    model = train_model(num_classes).to(device)

    # 使用预训练权重也可以是继续之前中断的训练但要配合log使用
    # 如果存在预训练权重则载入
    if args.model_weight_path != "":
        if os.path.exists(args.model_weight_path):
            weights_dict = torch.load(args.model_weight_path)
            model.load_state_dict(weights_dict, strict=False)
        else:
            raise FileNotFoundError("not found weights file: {}".format(args.weights))

    # log功能开发中
    # log... 需要保存的结果：模型参数量、flops、fps、mean_loss、accuracy
    # 检查是否有log文件夹 ../表示在父目录里找
    if not os.path.exists('../log'):
        # 没有就创建一个
        os.mkdir('../log')
    # 检查log下是否有当前模型对应的日志文件夹
    model_name = model.__class__.__name__
    if not os.path.exists(f'../log/{model_name}'):
        os.mkdir(f'../log/{model_name}')
    method_name = train_model.__name__
    if not os.path.exists(f'../log/{model_name}/{method_name}'):
        os.mkdir(f'../log/{model_name}/{method_name}')
    # 创建本次训练的日志文件夹 文件夹命名方式：train_数据集名称_年月日时分秒
    t = datetime.now() # 获取当前时间
    time_str = f'{t.year}-{t.month}-{t.day}_{t.hour}：{t.minute}：{t.second}'
    log_path = f'../log/{model_name}/{method_name}/train_{args.dataset_name}_{time_str}'
    os.mkdir(log_path)
    # 创建工作簿 模型性能、平均损失、top-1准确率
    wb = Workbook()
    performance_sheet = wb.create_sheet('performance')
    loss_sheet = wb.create_sheet('loss')
    accuracy_sheet = wb.create_sheet('accuracy')
    wb.remove(wb.active)
    sheet_names = wb.sheetnames
    wb.save(f'{log_path}/log.xlsx')

    # 是否冻结权重
    if args.freeze_layers:
        for name, para in model.named_parameters():
            # 除最后一个卷积层和全连接层外，其他权重全部冻结
            if ("features.top" not in name) and ("classifier" not in name):
                para.requires_grad_(False)
            else:
                print("training {}".format(name))

    # 各种设置的初始化
    # 在 PyTorch 中，CUDA 操作通常要求输入数据类型为 torch.float32
    # model.to(device=device, dtype=torch.float32)
    model = nn.DataParallel(model, device_ids=devices)
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs, eta_min=0.1 * args.lr)

    # train
    loss_sheet.append(['轮次', '平均损失'])
    accuracy_sheet.append(['轮次', 'top-1准确率'])
    loss_list = [[], []]
    accuracy_list = [[], []]
    best_accuracy = 0.0
    for epoch in range(args.epochs):
        # train
        model, mean_loss = train_one_epoch(model, train_dataloader, loss_function, optimizer, devices,
                                           scheduler, epoch, args.epochs)
        loss_sheet.append([epoch + 1, mean_loss])
        loss_list[0].append(epoch + 1)
        loss_list[1].append(mean_loss)

        # val
        if epoch % args.print_epoch == args.print_epoch - 1:
            accuracy = evaluate_one_epoch(model, test_dataloader, devices, epoch,
                                          args.epochs, args.eval_size)
            accuracy_sheet.append([epoch + 1, accuracy])
            accuracy_list[0].append(epoch + 1)
            accuracy_list[1].append(accuracy)
            # 模型保存
            if accuracy >= best_accuracy:
                best_accuracy = accuracy
                torch.save(model.state_dict(), f'{log_path}/{model_name}_best.pth')
        # save
        wb.save(f'{log_path}/log.xlsx')

    torch.save(model.state_dict(), f'{log_path}/{model_name}_finish.pth')

    # predict
    predict_accuracy, fps = predict_one_epoch(model, test_dataloader, devices)

    # 计算模型参数量、flops、fps
    test_input = torch.randn((1, 3, 224, 224), device=devices[0])
    model = train_model(num_classes).to(device=devices[0])
    flops, params = thop.profile(model, (test_input,))

    print('flops: %.2f M, params: %.2f M, fps: %.2f 张/s, accuracy: %.2f％' % (flops / 1e6, params / 1e6, fps, predict_accuracy * 100))
    # 保存于日志
    performance_sheet.append(['Flops(M)', '模型参数量(M)', 'fps(张/s)', 'top-1准确率(%)', 'batch_size', 'epochs', 'lr', 'num_classes'])
    performance_sheet.append([flops / 1e6, params / 1e6, fps, predict_accuracy, args.batch_size, args.epochs, args.lr, num_classes])
    wb.save(f'{log_path}/log.xlsx')

    # 将平均损失和准确率绘制成图
    # mean_loss
    show_png(loss_list, 'epoch', 'mean_loss', log_path)

    # mean_loss
    show_png(accuracy_list, 'epoch', 'accuracy', log_path)

In [3]:
def show_png(data_list, x, y, log_path):
    x_min, x_max = min(data_list[0]), max(data_list[0])
    y_min, y_max = min(data_list[1]), max(data_list[1])
    dx = x_max * 0.2
    dy = (y_max - y_min) * 0.2
    plt.xlim(0, x_max + dx)  # 横纵坐标上下限
    plt.ylim(max((y_min - dy), 0), y_max + dy + 0.1)

    plt.xlabel(f'{x}')  # 标签
    plt.ylabel(f'{y}')
    plt.xticks(list(range(0, int(x_max + dx), math.ceil(x_max / 10))))
    plt.plot(data_list[0], data_list[1], linewidth=2.0)  # 绘制曲线
    plt.title(f'{y}')
    plt.savefig(f'{log_path}/{y}.png')
    plt.show()
    plt.clf()

def predict_one_epoch(model, predict_dataloader, devices):
    model.eval()

    with torch.no_grad():
        true_predict_sum = 0
        predict_sum = 0
        compute_time_sum = 0
        predict_bar = tqdm(enumerate(predict_dataloader), desc='测试中...')
        for step, val_data in predict_bar:
            # 测试代码用
            if step == 2:
                break

            predict_bar.desc = f'测试中:第{step+1}/{len(predict_dataloader)}步'
            inputs, labels = val_data
            inputs, labels = inputs.to(device=devices[0]), labels.to(device=devices[0])

            # fps
            torch.cuda.synchronize()
            start = time.time()
            outputs = model(inputs)  # outputs:(batch_size, classes)
            torch.cuda.synchronize()
            end = time.time()
            compute_time_sum += (end - start)

            # torch.max返回是(max_value,max_index) index表示类
            predict = torch.max(outputs, dim=1)[1]  # predict:(batch_size,1)
            # 更新预测正确总数
            true_predict_sum += torch.eq(predict, labels).sum().cpu().item()
            # 更新预测总数
            predict_sum += labels.cpu().size(0)
        # 计算准确率
        accuracy = true_predict_sum / predict_sum
        # 计算fps
        fps = predict_sum / compute_time_sum
    return accuracy, fps

def evaluate_one_epoch(model, val_dataloader, devices, epoch, epochs, eval_size):
    model.eval()

    with torch.no_grad():
        true_predict_sum = 0
        predict_sum = 0
        val_bar = tqdm(enumerate(val_dataloader), desc=f'验证中:第{epoch+1}/{epochs}轮')
        for step, val_data in val_bar:
            # 测试代码用
            if step == eval_size:
                break

            val_bar.desc = f'验证中:第{epoch+1}/{epochs}轮,第{step+1}/{len(val_dataloader)}步'
            inputs, labels = val_data
            inputs, labels = inputs.to(device=devices[0]), labels.to(device=devices[0])

            outputs = model(inputs) # outputs:(batch_size, classes)
            torch.cuda.synchronize()

            # torch.max返回是(max_value,max_index) index表示类
            predict = torch.max(outputs, dim=1)[1]  # predict:(batch_size,1)
            # 更新预测正确总数
            true_predict_sum += torch.eq(predict, labels).sum().cpu().item()
            # 更新预测总数
            predict_sum += labels.cpu().size(0)
            # 计算准确率
        accuracy = true_predict_sum / predict_sum
        print(f'*****第{epoch+1}/{epochs}轮 accuracy:{accuracy}*****')
    return accuracy

def train_one_epoch(model, train_dataloader, loss_function, optimizer,
                    devices, scheduler, epoch, epochs):
    sum_loss = 0.0
    train_bar = tqdm(enumerate(train_dataloader), desc=f'训练中:第{epoch+1}/{epochs}轮训练中')
    for step, train_data in train_bar:
        # 测试代码用
        if step == 2:
            break

        train_bar.desc = f'训练中:第{epoch+1}/{epochs}轮,第{step+1}/{len(train_dataloader)}步'
        model.train()

        inputs, labels = train_data
        inputs, labels = inputs.to(device=devices[0]), labels.to(device=devices[0])

        # 梯度归零
        optimizer.zero_grad()

        # 前向传播
        outputs = model(inputs)
        torch.cuda.synchronize()

        # 计算损失
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        sum_loss += loss.cpu().item()

    mean_loss = sum_loss / len(train_dataloader)
    print(f'第{epoch+1}/{epochs}轮 mean_loss:{mean_loss} lr:{optimizer.param_groups[0]["lr"]}')

    # 学习率更新
    scheduler.step()

    return model, mean_loss

In [4]:
def parse_args(args=None):
    parser = argparse.ArgumentParser()

    # parser.add_argument('--dataset_root', type=str, default=r'D:\File\Data_Set\CIFAR\CIFAR-10')
    # parser.add_argument('--dataset_name', type=str, default='cifar-10')
    # parser.add_argument('--dataset_root', type=str, default=r'D:\File\Data_Set\CIFAR\CIFAR-100')
    # parser.add_argument('--dataset_name', type=str, default='cifar-100')
    # parser.add_argument('--dataset_root', type=str, default=r'D:\File\Data_Set\Caltech-256\Caltech-256-test')
    # parser.add_argument('--dataset_name', type=str, default='caltech-256')
    # parser.add_argument('--dataset_root', type=str, default=r'D:\File\Data_Set\MNIST')
    # parser.add_argument('--dataset_name', type=str, default='mnist')
    # parser.add_argument('--dataset_root', type=str, default=r'D:\File\Data_Set\Fashion-MNIST')
    # parser.add_argument('--dataset_name', type=str, default='fashion-mnist')
    # parser.add_argument('--dataset_root', type=str, default=r'D:\File\Data_Set\Oracle\oracle')
    # parser.add_argument('--dataset_name', type=str, default='oracle')
    parser.add_argument('--dataset_root', type=str, default=r'D:\File\Data_Set\Oracle\oracle-10')
    parser.add_argument('--dataset_name', type=str, default='oracle-10')

    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--epochs', type=int, default=4)
    parser.add_argument('--print_epoch', type=int, default=2)
    parser.add_argument('--eval_size', type=int, default=2)
    parser.add_argument('--num_gpus', type=int, default=1)

    # parser.add_argument('--dataset_root', type=str, default=r'/home/featurize/data')
    # parser.add_argument('--batch_size', type=int, default=64)
    # parser.add_argument('--epochs', type=int, default=100)
    # parser.add_argument('--print_epoch', type=int, default=5)
    # parser.add_argument('--eval_size', type=int, default=2)
    # parser.add_argument('--num_gpus', type=int, default=1)

    parser.add_argument('--model_weight_path', type=str, default=r'D:\File\python_project\image_detection _by_deep_learning\pytorch_classification\Experience01\log\MDT6\mdt20\train_oracle-10_2024-11-14_22：39：33\MDT6_finish.pth')
    parser.add_argument('--lr', type=float, default=1e-3)
    parser.add_argument('--device', type=str, default='cuda')
    parser.add_argument('--freeze_layers', type=bool, default=False)

    return parser.parse_args(args)

In [5]:
opt = parse_args([])
main(opt)

C:\Users\13397\AppData\Local\Temp\ipykernel_10872\2087122003.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights_dict = torch.load(args.model_weight_path)


Namespace(dataset_root='D:\\File\\Data_Set\\Oracle\\oracle-10', dataset_name='oracle-10', batch_size=64, epochs=4, print_epoch=2, eval_size=2, num_gpus=1, model_weight_path='D:\\File\\python_project\\image_detection _by_deep_learning\\pytorch_classification\\Experience01\\log\\MDT6\\mdt20\\train_oracle-10_2024-11-14_22：39：33\\MDT6_finish.pth', lr=0.001, device='cuda', freeze_layers=False)
使用设备：[device(type='cuda', index=0)]


训练中:第1/4轮,第2/83步: 2it [00:01,  1.43it/s]


第1/4轮 mean_loss:0.057027684636862884 lr:0.001


训练中:第2/4轮,第2/83步: 2it [00:01,  2.00it/s]


...

验证中:第4/4轮,第2/21步: 2it [00:00,  2.98it/s]


*****第4/4轮 accuracy:0.2109375*****


测试中:第2/21步: 2it [00:00,  3.26it/s]


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
flops: 3361.57 M, params: 10.26 M, fps: 990.54 张/s, accuracy: 20.31％


<Figure size 640x480 with 0 Axes>

In [6]:
! featurize instance release $UUID

'featurize' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
